In [ ]:
import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error



In [260]:
df = pd.read_csv("main_train_df.csv")#.iloc[0:100]
TARGET = "Sales"


In [91]:
cart_features = [
    "Customers",
    "Promo",
    "DayOfWeek",
    "SchoolHoliday",
    "Month",
]

cart_df = df[cart_features + [TARGET]]


In [92]:
def variance(rows):
    return np.var(rows[:, -1])

def variance_reduction(left, right, current_var):
    n = len(left) + len(right)
    return current_var - (
        len(left)/n * variance(left) + len(right)/n * variance(right)
    )

In [93]:
class Question:
    def __init__(self, column, value, feature_names):
        self.column = column
        self.value = value
        self.feature_names = feature_names

    def match(self, row):
        return row[self.column] >= self.value

    def __repr__(self):
        return f"Is {self.feature_names[self.column]} >= {self.value}?"


In [94]:
def partition(rows, question):
    mask = rows[:, question.column] >= question.value
    return rows[mask], rows[~mask]


In [95]:
def find_best_split(rows, feature_names):
    best_gain = 0
    best_question = None
    current_var = variance(rows)
    n_features = rows.shape[1] - 1

    for col in range(n_features):
        values = np.unique(rows[:, col])
        for val in values:
            q = Question(col, val, feature_names)
            left, right = partition(rows, q)
            if len(left) == 0 or len(right) == 0:
                continue
            gain = variance_reduction(left, right, current_var)
            if gain > best_gain:
                best_gain, best_question = gain, q

    return best_gain, best_question


In [96]:
class Leaf:
    def __init__(self, rows):
        self.value = rows[:, -1].mean()
        self.n_samples = len(rows)

class DecisionNode:
    def __init__(self, question, left, right):
        self.question = question
        self.left = left
        self.right = right


In [97]:
class LeafCounter:
    def __init__(self, max_leaves):
        self.max_leaves = max_leaves
        self.count = 0


In [98]:
def build_tree(rows, feature_names, counter, min_samples_leaf=1000):
    gain, question = find_best_split(rows, feature_names)

    if gain == 0 or len(rows) <= min_samples_leaf or counter.count >= counter.max_leaves:
        counter.count += 1
        return Leaf(rows)

    left, right = partition(rows, question)

    if counter.count + 2 > counter.max_leaves:
        counter.count += 1
        return Leaf(rows)

    left_branch = build_tree(left, feature_names, counter, min_samples_leaf)
    right_branch = build_tree(right, feature_names, counter, min_samples_leaf)

    return DecisionNode(question, left_branch, right_branch)


In [99]:
cart_array = cart_df.to_numpy()
counter = LeafCounter(max_leaves=5)

cart_tree = build_tree(
    cart_array,
    cart_features,
    counter,
    min_samples_leaf=1000  # adjust for dataset size
)


In [100]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing + f"Leaf: value={node.value:.2f}, samples={node.n_samples}")
        return

    print(spacing + str(node.question))
    print(spacing + "--> True:")
    print_tree(node.left, spacing + "  ")
    print(spacing + "--> False:")
    print_tree(node.right, spacing + "  ")

print_tree(cart_tree)


Is Customers >= 425?
--> True:
  Is Customers >= 966?
  --> True:
    Is Customers >= 1669?
    --> True:
      Is Customers >= 2901?
      --> True:
        Is Customers >= 4051?
        --> True:
          Leaf: value=27369.41, samples=230
        --> False:
          Is Promo >= 1?
          --> True:
            Is Month >= 12?
            --> True:
              Leaf: value=24216.88, samples=144
            --> False:
              Is DayOfWeek >= 2?
              --> True:
                Is Customers >= 3527?
                --> True:
                  Leaf: value=19962.00, samples=271
                --> False:
                  Leaf: value=18378.68, samples=976
              --> False:
                Leaf: value=20886.15, samples=365
          --> False:
            Leaf: value=17196.19, samples=1861
      --> False:
        Leaf: value=14749.33, samples=17154
    --> False:
      Leaf: value=10141.90, samples=111549
  --> False:
    Leaf: value=6388.98, samples=463355
--> Fa

In [101]:
leaf_id_map = {}

def get_leaf_id(node, row):
    if isinstance(node, Leaf):
        if node not in leaf_id_map:
            leaf_id_map[node] = len(leaf_id_map)  # 0,1,2,3,4
        return leaf_id_map[node]
    if node.question.match(row):
        return get_leaf_id(node.left, row)
    else:
        return get_leaf_id(node.right, row)

In [102]:
df["leaf_id"] = [
    get_leaf_id(cart_tree, row)
    for row in cart_array
]

print(df["leaf_id"].unique())

[0 1 2 3 4 5 6 7 8 9]


In [103]:
df[df["leaf_id"] == 0]

,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day,WeekOfYear,Sales,leaf_id
0,44,2,763,1,1,0,0,a,a,540.0,...,2011.0,0,NaN,NaN,2014,5,6,19,7076,0
1,346,2,663,1,1,0,1,a,c,8090.0,...,NaN,0,NaN,NaN,2014,7,29,31,8129,0
5,630,5,798,1,1,0,1,a,a,1690.0,...,2015.0,0,NaN,NaN,2013,8,16,33,6812,0
7,201,3,488,1,1,0,1,d,a,20260.0,...,NaN,1,18.0,2014.0,2013,7,31,31,6203,0
8,1096,3,623,1,1,0,0,a,c,1130.0,...,NaN,1,10.0,2014.0,2015,1,28,5,6292,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813758,857,5,928,1,1,0,0,c,a,6270.0,...,2005.0,1,23.0,2014.0,2015,3,20,12,7914,0
813759,932,6,535,1,0,0,0,a,a,15700.0,...,NaN,1,13.0,2010.0,2013,10,5,40,4994,0
813761,135,2,703,1,1,0,1,d,a,5190.0,...,NaN,1,1.0,2013.0,2015,3,31,14,9776,0
813762,923,4,609,1,0,0,0,a,a,280.0,...,2008.0,0,NaN,NaN,2015,3,12,11,4790,0


In [107]:
leaf_models = {}
for leaf_id, leaf_data in df.groupby("leaf_id"):
    print(f"Training AutoGluon for leaf {leaf_id}, samples = {len(leaf_data)}")

    # Drop leaf_id from training features
    predictor = TabularPredictor(label=TARGET, eval_metric="rmse").fit(
        leaf_data.drop(columns=["leaf_id"]),
        presets="best_quality",  # or "medium_quality_faster_train"
        time_limit=600  # optional: seconds per leaf
    )

    # Store the model
    leaf_models[leaf_id] = predictor


No path specified. Models will be saved in: "AutogluonModels/ag-20260107_170106"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.11
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.4.0: Fri Mar 15 00:19:22 PDT 2024; root:xnu-10063.101.17~1/RELEASE_ARM64_T8112
CPU Count:          8
Memory Avail:       1.74 GB / 8.00 GB (21.8%)
Disk Space Avail:   52.80 GB / 228.27 GB (23.1%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as

Training AutoGluon for leaf 0, samples = 463355


2026-01-07 18:01:08,449	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2026-01-07 18:01:11,358	ERROR services.py:1350 -- Failed to start the dashboard , return code 0
2026-01-07 18:01:11,360	ERROR services.py:1375 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#logging-directory-structure' to find where the log file is.
2026-01-07 18:01:11,374	ERROR services.py:1419 -- 
The last 20 lines of /tmp/ray/session_2026-01-07_18-01-08_511886_16943/logs/dashboard.log (it contains the error message from the dashboard): 
    loop.run_until_complete(dashboard.r

Training AutoGluon for leaf 1, samples = 217859


Leaderboard on holdout data (DyStack):
                 model  score_holdout   score_val              eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L3    -151.950606 -172.239485  root_mean_squared_error       15.410714     248.270349  109.841872                 0.002713                0.004021           0.087706            3       True          4
1    LightGBMXT_BAG_L1    -152.384332 -172.813928  root_mean_squared_error       12.268602     207.807185   80.755599                12.268602              207.807185          80.755599            1       True          1
2  WeightedEnsemble_L2    -152.384332 -172.813928  root_mean_squared_error       12.271234     207.809898   80.760855                 0.002632                0.002713           0.005256            2       True          2
3    LightGBMXT_BAG_L2    -153.047014 -174.489321  root_mean_squared_error   

...

Leaderboard on holdout data (DyStack):
                          model  score_holdout    score_val              eval_metric  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         NeuralNetTorch_BAG_L2   -1327.749383  -976.183569  root_mean_squared_error        0.482246       1.157234  33.731328                 0.045998                0.289531          11.237407            2       True         19
1           WeightedEnsemble_L3   -1328.616916  -968.697467  root_mean_squared_error        0.548385       1.303832  35.553396                 0.001219                0.000325           0.008049            3       True         21
2             LightGBMXT_BAG_L2   -1330.674176 -1029.842464  root_mean_squared_error        0.463160       0.915751  23.876967                 0.026912                0.048048           1.383047            2       True         13
3          LightGBM_r131_BAG_L1   -1402.8

Training AutoGluon for leaf 8, samples = 271


Leaderboard on holdout data (DyStack):
                        model  score_holdout    score_val              eval_metric  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      NeuralNetFastAI_BAG_L1   -1060.681701 -1300.047585  root_mean_squared_error        0.470514       0.365240   9.425425                 0.470514                0.365240           9.425425            1       True          5
1           LightGBMXT_BAG_L2   -1144.259533 -1083.213287  root_mean_squared_error        1.478713       1.089096  27.662337                 0.082262                0.058465           2.015279            2       True         11
2             LightGBM_BAG_L2   -1156.690213 -1106.074755  root_mean_squared_error        1.429199       1.067459  26.847542                 0.032748                0.036828           1.200484            2       True         12
3           LightGBMXT_BAG_L1   -1175.386588 -111

Training AutoGluon for leaf 9, samples = 230


Leaderboard on holdout data (DyStack):
                        model  score_holdout    score_val              eval_metric  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       NeuralNetTorch_BAG_L1   -1393.831348 -1743.870958  root_mean_squared_error        0.062262       0.242067  18.050378                 0.062262                0.242067          18.050378            1       True          7
1        ExtraTreesMSE_BAG_L1   -1432.209876 -1780.306917  root_mean_squared_error        0.051355       0.092816   0.390013                 0.051355                0.092816           0.390013            1       True          4
2   NeuralNetTorch_r79_BAG_L1   -1492.014695 -1684.416931  root_mean_squared_error        0.068652       0.421702  16.823661                 0.068652                0.421702          16.823661            1       True          9
3              XGBoost_BAG_L2   -1516.827149 -195

In [155]:
leaf_models

{0: <autogluon.tabular.predictor.predictor.TabularPredictor at 0x344ee4090>,
 1: <autogluon.tabular.predictor.predictor.TabularPredictor at 0x174efc090>,
 2: <autogluon.tabular.predictor.predictor.TabularPredictor at 0x37781a490>,
 3: <autogluon.tabular.predictor.predictor.TabularPredictor at 0x310719a10>,
 4: <autogluon.tabular.predictor.predictor.TabularPredictor at 0x344efe8d0>,
 5: <autogluon.tabular.predictor.predictor.TabularPredictor at 0x344ee50d0>,
 6: <autogluon.tabular.predictor.predictor.TabularPredictor at 0x311083150>,
 7: <autogluon.tabular.predictor.predictor.TabularPredictor at 0x3110fa290>,
 8: <autogluon.tabular.predictor.predictor.TabularPredictor at 0x344ee5d10>,
 9: <autogluon.tabular.predictor.predictor.TabularPredictor at 0x36849fb90>}

In [223]:
def predict(df_new):
    preds = []
    leaves = []
    cart_array_new = df_new[cart_features].to_numpy()
    for i, row in enumerate(cart_array_new):
        # Find which leaf this row belongs to
        leaf = get_leaf_id(cart_tree, row)
        leaves.append(leaf)
        model = leaf_models[leaf]
        pred = model.predict(df_new.iloc[[i]]).iloc[0]  # single row prediction
        preds.append(pred)
    return np.array(preds) , np.array(leaves)


## Performance comparison

In [262]:
'''
global_predictor = TabularPredictor(
        label="Sales",
        path=f"new_global/" ).fit(
        df,
        presets="best",
        time_limit=600,)
'''

Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.11
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.4.0: Fri Mar 15 00:19:22 PDT 2024; root:xnu-10063.101.17~1/RELEASE_ARM64_T8112
CPU Count:          8
Memory Avail:       1.62 GB / 8.00 GB (20.2%)
Disk Space Avail:   41.04 GB / 228.27 GB (18.0%)
Presets specified: ['best']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to i

In [274]:
main_test_df = pd.read_csv("main_test_df.csv")
test_df_1 = main_test_df.iloc[0:1000]
test_df_2 = main_test_df.sample(frac=0.01, random_state=42)


### Smaple 1

In [ ]:
# CART algo
df_new = test_df_1.copy()
preds_new , leaves= predict(df_new)
df_new["predicted_sales"] = preds_new
df_new["leaf"] = leaves
df_new

SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [282]:
y_true = test_df_1["Sales"]
y_pred = df_new["predicted_sales"]
rmse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
print(f"rmse: {rmse}")
print(f"mae: {mae}")

rmse: 107184.8828125
mae: 225.50997924804688


In [283]:
# Global model 
global_predictor = TabularPredictor.load(f"new_global/")
y_pred = global_predictor.predict(test_df_1)
y_true = test_df_1["Sales"]
rmse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true , y_pred)
print(f"rmse: {rmse}")
print(f"mae: {mae}")

rmse: 205480.359375
mae: 299.8565673828125


### Sample 2

In [284]:
# CART algo
df_new = test_df_2.copy()
preds_new , leaves= predict(df_new)
df_new["predicted_sales"] = preds_new
df_new["leaf"] = leaves
df_new

,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day,WeekOfYear,Sales,predicted_sales,leaf
138619,438,1,630,1,1,0,0,d,c,1110.0,...,1,40.0,2012.0,2013,4,22,17,6750,6418.458008,0
39765,87,1,915,1,0,0,0,a,a,560.0,...,0,NaN,NaN,2013,5,6,19,6734,6690.619141,0
173119,1044,6,1224,1,0,0,0,c,a,240.0,...,1,13.0,2010.0,2013,12,14,50,10141,9059.969727,2
53208,602,7,0,0,0,0,0,a,a,2710.0,...,1,22.0,2012.0,2014,3,2,9,0,1.005319,1
142984,332,7,0,0,0,0,0,a,a,1840.0,...,0,NaN,NaN,2014,10,26,43,0,-0.445089,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52523,413,2,1177,1,1,0,0,a,c,760.0,...,0,NaN,NaN,2013,4,30,18,10051,9842.345703,2
176470,84,7,0,0,0,0,0,a,c,11810.0,...,0,NaN,NaN,2013,6,16,24,0,0.400811,1
60831,431,3,775,1,0,0,0,d,c,4520.0,...,0,NaN,NaN,2014,6,25,26,6662,7185.153809,0
7698,781,1,512,1,0,0,1,a,a,630.0,...,0,NaN,NaN,2015,2,9,7,3382,3581.304688,0


In [285]:
y_true = test_df_2["Sales"]
y_pred = df_new["predicted_sales"]
rmse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
print(f"rmse: {rmse}")
print(f"mae: {mae}")


rmse: 135673.171875
mae: 224.81146240234375


In [286]:
# Global model 
y_pred = global_predictor.predict(test_df_2)
y_true = test_df_2["Sales"]
rmse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true , y_pred)
print(f"rmse: {rmse}")
print(f"mae: {mae}")

rmse: 217444.984375
mae: 289.9985046386719
